# Data manipulation in Julia

In this session, we will cover the basics of data manipulation in Julia, including
- reading
- merging/joining
- summarizing/grouping
- plotting

At the end of the class, you should be able to construct these plots:

<table>
<tr>
<td> <img src="figures/n_loglik_val_A.pdf" style="width: 350px;"/></td>
<td> <img src="figures/n_loglik_val_F.pdf" style="width: 350px;"/></td>
</tr></table>

We study the sparse inverse covariance problem and compare 3 methods: 
- Meinshausena and Buhlmann's approximation (BM)
- Graphical Lasso (Glasso)
- Discrete formulation (Big-M)

We design experiments with 
- fixed dimension $p=200$
- fixed underlying true sparsity $k_{true}=199$
- varying number of samples $n$ (12 different values)

We run each experiment $10$ times and want to report average results over all simulations. 

## Step 1: Read the data

We generate one CSV file per method (3) per experiment (12) and per simulation (10) !

Let us look at the first CSV file

In [1]:
using DataFrames, CSV

In [ ]:
?CSV.read

In [2]:
df = CSV.read("experiment/baseline_BM_0.csv")

,run,p,n,ktrue,lambda,k,time,objval,EBIC,loglik_val,loglik_test,TF,FF,operatorNorm,frobeniusNorm,L1MatrixNorm,LInfVectorNorm
,Int64⍰,Int64⍰,Int64⍰,Int64⍰,Float64⍰,Int64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰
1,0,200,50,199,0.650391,50,0.339093,300.028,233.752,231.957,228.719,296.0,4.0,2.49816,16.4842,3.93596,0.935962
2,0,200,50,199,0.541016,100,0.353749,285.591,238.208,221.608,218.917,384.0,16.0,2.37917,15.749,3.83156,0.892957
3,0,200,50,199,0.460938,149,0.358401,273.489,242.102,211.237,208.907,456.0,42.0,2.25852,14.9471,3.72804,0.857387
4,0,200,50,199,0.422852,199,0.55993,267.044,250.186,205.402,203.288,490.0,106.0,2.18994,14.4682,3.65937,0.839065
5,0,200,50,199,0.385742,249,0.547767,260.111,257.355,199.16,197.26,512.0,184.0,2.1109,13.933,3.57429,0.820244
6,0,200,50,199,0.36499,298,0.802488,255.907,266.587,195.478,193.785,526.0,266.0,2.058,13.5981,3.51101,0.809273
7,0,200,50,199,0.347534,348,0.840247,252.135,276.116,192.289,190.692,536.0,352.0,2.0072,13.296,3.45975,0.799783
8,0,200,50,199,0.329712,398,0.855924,248.049,285.062,188.999,187.452,542.0,442.0,1.92268,12.9505,3.32232,0.789837


In [3]:
size(df)

(8, 17)

Let us have a look at the columns

In [4]:
names(df)

17-element Array{Symbol,1}:
 :run           
 :p             
 :n             
 :ktrue         
 :lambda        
 :k             
 :time          
 :objval        
 :EBIC          
 :loglik_val    
 :loglik_test   
 :TF            
 :FF            
 :operatorNorm  
 :frobeniusNorm 
 :L1MatrixNorm  
 :LInfVectorNorm

The first 4 columns identify the experiment and the simulation
 - :run is the number of the simulation (between 0 and 9)          
 - :p is the dimension of the problem             
 - :n is the number of samples            
 - :ktrue is the true sparsity pattern 

They take the same value for all rows in the file.

The other columns correspond to the regularization path:
 - :lambda is the value of the regularization paramter       
 - :k is the sparsity of the corresponding estimator      
 - :time is the time needed to compute the solution       
 - :objval is the in-sample objective value

Then, we report performance metrics that can be used for cross-validation:
 - :EBIC is the value of the in-sample Extended Bayesian Information Criterion (the lower the better)
 - :loglik_val is the value of the negative log-likelihood on the validation set (the lower the better)

Finally, we report final metrics on the test set
 - :loglik_test is the negative log-likelihood on the test test
 - :TF is the number of true features selected          
 - :FF is the number of false features selected          

We will not use the final 4 columns

**Side note:** Renaming columns

In [ ]:
DataFrames.rename(df, (:operatorNorm => :uselessColumn))

But did not change the original table df

In [ ]:
df

Suffix "!" denotes functions which modify their argument (convention only)

In [ ]:
DataFrames.rename!(df, (:operatorNorm => :uselessColumn))
df

>**\[Exercise\]**: Read/Concatenate 

> There are 120 files for the method "BM", write a Julia function which reads and concatenates all the CSV files into one dataframe

> Note: you can concatenate dataframes horizontally (preserves the first dimension) or vertically (preserves the second dimension) using hcat(df1, df2) or vcat(df1, df2) respectively

In [10]:
"experiment/baseline_BM_"*"3"

"experiment/baseline_BM_3"

In [24]:
function merge_files(prefix)
    #write your code here
    filename = prefix*string(0)*".csv"
    bigdf = CSV.read(filename)
    
    for file_number in 1:119
#         filename = prefix*string(file_number)*".csv"
        filename = string(prefix, file_number, ".csv")
        df = CSV.read(filename)
        bigdf = vcat(bigdf,df)
    end
    
    return bigdf
end

merge_files (generic function with 1 method)

In [25]:
bm = merge_files("experiment/baseline_BM_")

,run,p,n,ktrue,lambda,k,time,objval,EBIC,loglik_val,loglik_test,TF,FF,operatorNorm,frobeniusNorm,L1MatrixNorm,LInfVectorNorm
,Int64⍰,Int64⍰,Int64⍰,Int64⍰,Float64⍰,Int64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰
1,0,200,50,199,0.650391,50,0.339093,300.028,233.752,231.957,228.719,296.0,4.0,2.49816,16.4842,3.93596,0.935962
2,0,200,50,199,0.541016,100,0.353749,285.591,238.208,221.608,218.917,384.0,16.0,2.37917,15.749,3.83156,0.892957
3,0,200,50,199,0.460938,149,0.358401,273.489,242.102,211.237,208.907,456.0,42.0,2.25852,14.9471,3.72804,0.857387
4,0,200,50,199,0.422852,199,0.55993,267.044,250.186,205.402,203.288,490.0,106.0,2.18994,14.4682,3.65937,0.839065
5,0,200,50,199,0.385742,249,0.547767,260.111,257.355,199.16,197.26,512.0,184.0,2.1109,13.933,3.57429,0.820244
6,0,200,50,199,0.36499,298,0.802488,255.907,266.587,195.478,193.785,526.0,266.0,2.058,13.5981,3.51101,0.809273
7,0,200,50,199,0.347534,348,0.840247,252.135,276.116,192.289,190.692,536.0,352.0,2.0072,13.296,3.45975,0.799783
8,0,200,50,199,0.329712,398,0.855924,248.049,285.062,188.999,187.452,542.0,442.0,1.92268,12.9505,3.32232,0.789837
9,1,200,50,199,0.585938,50,0.382404,291.702,228.96,219.76,220.356,292.0,8.0,2.40384,16.4579,3.92658,0.939546


In [23]:
size(bm)

(960, 17)

## Step 2: Define relevant columns

In the end, we want to represent accuracy ($A$) and false detection rate ($F$) as a funtion of $n/p$

>**\[Exercise\]**: Add columns

> Write a Julia function which takes a dataframe and add three columns: n/p, A and F.

> Note:
$$ A = \dfrac{TF - p}{2 k_{true}}, \quad F = \dfrac{FF}{2 k} $$

In [ ]:
function add_columns!(df)
    df[:ntop] = df[:n] ./ df[:p]
    df[:A] = df[:TF] .- p ./ 2 ./2 df[:k_true]
    #write your code here
end

In [ ]:
add_columns!(bm)
bm

## Step 3: Perform the CV

For each experiment and simulation, we need to select the regularization parameter which minimizes the cross-validation criterion (either :EBIC or :loglik_val)

We need to use a "group by" strategy

In [ ]:
?DataFrames.groupby

The command

In [ ]:
gr = DataFrames.groupby(bm, [:run, :p, :n, :ktrue])

creates a "list of dataframes", corresponding to each group.

In [ ]:
length(gr)

You can iterate over it

In [ ]:
for subgroup in gr
    println(size(subgroup))
end

>**\[Exercise\]**: Group by/Concatenate 

> Write a Julia function which takes as arguments
- a dataframe, 
- the list of columns which define an experiment, 
- the column of the cross-validation criterion 

> and performs the cross validation for all experiments.

In [ ]:
function perform_cv(df, id_cols, crit_col)
    #write your code here
end    

In [ ]:
bm_cv = perform_cv(bm, [:run, :p, :n, :ktrue], :loglik_val)

In [ ]:
size(bm_cv)

## Step 4: Aggregate results

For each experiment, we want to compute average accuracy and false detection (and their corresponding standard deviation)

>**\[Exercise\]**: Group by/Concatenate 

> Use a "group by" syntax to write a Julia function which takes as arguments
- a dataframe, 
- the list of columns which define an experiment, 

> and returns a dataframe with average/standard deviation for A.

In [ ]:
using Statistics
function aggregate_A(df, exp_cols)
    #write your code here
end

In [ ]:
aggregate_A(bm_cv, [:p, :n, :ktrue])

This is already the second time we are writing a code like this: iterate over all subgroups, perform some data manipulations and concatenate results from each subgroups. There must be a better way! 

Indeed...

In [ ]:
by(df, cols) do aux #Similar as "for aux in groupby(df, cols)"
    #write all operations you want to do
    #last line should be a dataframe
end

This is refered to as the **split-apply-combine** strategy

>**\[Exercise\]**: "Group by/Concatenate in one shot" 

> Rewrite aggregate_A using a "by(df, col) do ..." syntax

In [ ]:
using Statistics
function aggregate_A2(df, exp_cols)
    #write your code here
end

In [ ]:
aggregate_A2(bm_cv, [:p, :n, :ktrue])

NB: Notice that this syntax always adds the columns you grouped on.

Yet, we still had to manually create columns. It can be tedious if you want to look at multiple outputs (A, F, time, loglik_test,...). There must be a better way!

Indeed...

In [ ]:
?DataFrames.aggregate

>**\[Exercise\]**: "Aggregate" 

> Rewrite aggregate_A2 using the aggregate function

In [ ]:
using Statistics
function aggregate_A3(df, exp_cols)
    #write your code here
end

In [ ]:
aggregate_A3(bm_cv, [:p, :n, :ktrue])

>**\[Exercise\]**: "Split-apply-combine" strategy 

> Write a Julia function which takes a dataframe and returns summary statistics over all simulations for A, F, time and loglik_test

In [ ]:
using Statistics
function aggregate_experiment(df, exp_cols)
    by(df, exp_cols) do aux
        aggregate(aux[:,[:ntop, :A, :F, :time]], :ntop, [mean, std])
    end
end

In [ ]:
aggregate_experiment(bm_cv, [:p, :n, :ktrue])

## Step 5: Upload and process results for all methods

In [ ]:
bm = merge_files("experiment/baseline_BM_") #1
add_columns!(bm) #2
bm_cv = perform_cv(bm, [:run, :p, :n, :ktrue], :loglik_val) #3
bm_path = aggregate_experiment(bm_cv, [:p, :n, :ktrue]) #4

glasso = merge_files("experiment/baseline_Glasso_")
add_columns!(glasso)
glasso_cv = perform_cv(glasso, [:run, :p, :n, :ktrue], :loglik_val)
glasso_path = aggregate_experiment(glasso_cv, [:p, :n, :ktrue])

bigm = merge_files("experiment/n_bigm_")
add_columns!(bigm)
bigm_cv = perform_cv(bigm, [:run, :p, :n, :ktrue], :loglik_val)
bigm_path = aggregate_experiment(bigm_cv, [:p, :n, :ktrue])

## Step 6: Plot

Julia has a very nice plot package [**Plots.jl**](https://juliaplots.github.io). Plots.jl is an interface which sits on top of many backends.

In [ ]:
using Plots

In [ ]:
Plots.pyplot() #pyplot backend
Plots.plot(bm_path[:ntop], bm_path[:A_mean], yerr=bm_path[:A_std])

In [ ]:
Plots.gr() #GR backend (default)
Plots.plot(bm_path[:ntop], bm_path[:A_mean], yerr=bm_path[:A_std], label="MB", xaxis="n/p")

Use the plot! function to modify the current plot.

In [ ]:
Plots.plot!(bm_path[:ntop], bm_path[:F_mean], yerr=bm_path[:F_std])

>**\[Exercise\]**: Plotting 

> Show on one graph how the accuracy $A$ evolves as $n/p$ increases for all methods

In [ ]:
#write your code here

**Bonus: ** 

- Other syntax using the StatPlots package: [**StatPlots.jl**](https://github.com/JuliaPlots/StatPlots.jl) implements some useful recipes in data analysis and statistics
- You can use LaTeX text in your legends using [**LaTeXStrings.jl**](https://github.com/stevengj/LaTeXStrings.jl)

In [ ]:
using StatPlots, LaTeXStrings

In [ ]:
@df bm_path StatPlots.plot(:ntop, :A_mean, yerr=:A_std, label="MB")
@df glasso_path StatPlots.plot!(:ntop, :A_mean, yerr=:A_std, label="Glasso")
@df bigm_path StatPlots.plot!(:ntop, :A_mean, yerr=:A_std, label="Big-M")
xaxis!(L"n/p")
yaxis!(L"A")

It can be tedious to write a line for each method specifically. It would be more efficient to:
- add a column method to each dataframe
- merge the three dataframes into one
- use the method column as a group indicator

In [ ]:
bm_path[:method] = "MB"
glasso_path[:method] = "Glasso"
bigm_path[:method] = "Big-M"

In [ ]:
big_df = vcat(bm_path, glasso_path)
big_df = vcat(big_df, bigm_path)

In [ ]:
@df big_df StatPlots.plot(:ntop, :F_mean, yerr=:F_std, group=:method, line=1)
xaxis!(L"n/p")
yaxis!(L"F")

## Bonus: Interoperability and RCall

Julia has a growing number of packages implementing basic up to state-of-the-art statistics/ML techniques (e.g. GLM, GLMnet, LibLinear, ...) but is still a young programming language. 

**Good news!** If you ever need to, you can easily interact directly with R or Python

In [ ]:
using RCall

You can easily write and compile R code with the Rstring syntaxt

In [ ]:
R"1+2"

You can use Julia objects in the R code using the "$" prefix

Note: avoid using special characters for variable names

In [ ]:
R"$big_df"

So if you
- do not know how to do something in Julia
- do know how to do it in R 
- are lazy to go from one language into another 

Simply use RCall!

In [ ]:
R"""
library(dplyr)
library(ggplot2)

$big_df %>% 
    ggplot()+aes(x=ntop, y=F_mean, color=factor(method)) + geom_line()
"""

## Bonus: Saving Julia objects using 

JLD/JLD2 enables to save/load Julia objects very easily (similar to pickle for Python)

In [ ]:
using RCall

In [ ]:
@save "test.jld2" big_df

In [ ]:
@load "test.jld2" big_df